In [387]:
import yfinance as yf
import pandas as pd
ticker = "AAPL"
start_date = "2015-01-01"
end_date = "2025-11-01"

df = yf.download(ticker, start=start_date, end=end_date)
df = df.dropna()
df.head()


C:\Users\toors\AppData\Local\Temp\ipykernel_3056\2289571361.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2015-01-02,24.237549,24.705318,23.798599,24.694233,212818400
2015-01-05,23.554745,24.086805,23.368524,24.006996,257142000
2015-01-06,23.556961,23.816340,23.195602,23.619034,263188400
2015-01-07,23.887281,23.987042,23.654504,23.765350,160423600
2015-01-08,24.805088,24.862728,24.097891,24.215389,237458000


In [388]:
df.columns=['_'.join(col).strip() for col in df.columns.values]

In [389]:
df.head()

,Close_AAPL,High_AAPL,Low_AAPL,Open_AAPL,Volume_AAPL
Date,,,,,
2015-01-02,24.237549,24.705318,23.798599,24.694233,212818400
2015-01-05,23.554745,24.086805,23.368524,24.006996,257142000
2015-01-06,23.556961,23.816340,23.195602,23.619034,263188400
2015-01-07,23.887281,23.987042,23.654504,23.765350,160423600
2015-01-08,24.805088,24.862728,24.097891,24.215389,237458000


In [390]:
apple_data.tail()

,Close_AAPL,High_AAPL,Low_AAPL,Open_AAPL,Volume_AAPL,z_score
Date,,,,,,
2025-11-17,267.459991,270.489990,265.730011,268.820007,45018300,2.235848
2025-11-18,267.440002,270.709991,265.320007,269.989990,45677300,2.235574
2025-11-19,268.559998,272.209991,265.500000,265.529999,40424500,2.250960
2025-11-20,266.250000,275.429993,265.920013,270.829987,45823600,2.219225
2025-11-21,271.489990,273.329987,265.670013,265.950012,59030800,2.291212


In [391]:
#daily price movement
df["returns"] = df["Close_AAPL"].pct_change()


In [392]:
df.head()

,Close_AAPL,High_AAPL,Low_AAPL,Open_AAPL,Volume_AAPL,returns
Date,,,,,,
2015-01-02,24.237549,24.705318,23.798599,24.694233,212818400,NaN
2015-01-05,23.554745,24.086805,23.368524,24.006996,257142000,-0.028171
2015-01-06,23.556961,23.816340,23.195602,23.619034,263188400,0.000094
2015-01-07,23.887281,23.987042,23.654504,23.765350,160423600,0.014022
2015-01-08,24.805088,24.862728,24.097891,24.215389,237458000,0.038422


In [393]:
#monthly volatility
df["volatility_20"] = df["returns"].rolling(20).std()

In [394]:
#Rolling Z_score
df["price_z_90"]= (df["Close_AAPL"]- df["Close_AAPL"].rolling(90).mean())/df["Close_AAPL"].std()


In [395]:
# momentum(short term trend strength)
df["momentum_30"] = df["Close_AAPL"].pct_change(30)


In [396]:
#ATR-14days
df["previous_close"]= df["Close_AAPL"].shift(1)
df["tr1"]= df["High_AAPL"]- df["Low_AAPL"]
df["tr2"]= (df["High_AAPL"]- df["previous_close"]).abs()
df["tr3"]= (df["Low_AAPL"]- df["previous_close"]).abs()
df["TR"] = df[["tr1", "tr2", "tr3"]].max(axis=1)
df["ATR_14"]= df["TR"].rolling(14).mean()


In [397]:
df=df.drop(columns=['previous_close','tr1','tr2','tr3'])

In [398]:
df.dropna(inplace=True)

In [399]:
#creating target variable
df["vol_fut"] = df["volatility_20"].shift(-1)
df["risk_class"] = pd.qcut(df["vol_fut"], 3, labels=[0, 1, 2])

In [400]:
df.drop(columns=["vol_fut"],inplace=True)
df.dropna(inplace=True)

In [426]:
X= df.drop(columns=["risk_class"])
y= df["risk_class"]

In [428]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=42, shuffle= False)

In [430]:
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier(n_estimators=300, max_depth=6)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, n_estimators=300)

In [444]:
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
y_pred= model.predict(X_test)
cm=confusion_matrix(y_test, y_pred)

In [440]:
cm

array([[101,  98,   1],
       [  3, 183,   6],
       [  1,  15, 119]], dtype=int64)

In [442]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.96      0.51      0.66       200
           1       0.62      0.95      0.75       192
           2       0.94      0.88      0.91       135

    accuracy                           0.76       527
   macro avg       0.84      0.78      0.77       527
weighted avg       0.83      0.76      0.76       527



In [446]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.7647058823529411


In [453]:
import xgboost as xgb
model = xgb.XGBClassifier(
    n_estimators=300,        # number of trees
    max_depth=6,             # max depth of each tree
    learning_rate=0.1,       # shrinkage for each tree
    objective='multi:softmax', # multi-class classification
    num_class=3,             # number of classes
    random_state=42,
    eval_metric='mlogloss'
)
model.fit(X_train, y_train)

# 5️⃣ Predictions
y_pred = model.predict(X_test)

# 6️⃣ Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 7️⃣ Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

Accuracy: 0.8671726755218216

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.83      0.89       200
           1       0.78      0.90      0.84       192
           2       0.91      0.87      0.89       135

    accuracy                           0.87       527
   macro avg       0.88      0.87      0.87       527
weighted avg       0.88      0.87      0.87       527

